In [46]:
#!pip install -r requirements.txt
import math as m
import numpy as np
import abc

from tqdm import tqdm

In [47]:
class Dataset:
    def __init__():
        pass

In [48]:
def TTV_split(data, tr, te, va, verbose=False):
    total = len(data)
    one = total / (tr + te + va)

    tr_n = m.floor(tr * one)
    te_n = m.floor(te * one)
    va_n = m.floor(va * one)

    tr_n += (total - tr_n - te_n - va_n)

    train_data = np.array([])
    test_data = np.array([])
    val_data = np.array([])

    np.random.shuffle(data)

    for i in tqdm(data):
        if (len(train_data) < tr_n):
            np.append(train_data, i)
            continue
        if (len(test_data) < te_n):
            np.append(test_data, i)
            continue
        if (len(val_data) < va_n):
            np.append(val_data, i)

    if verbose:
        print(f"total = {total}\ntr_n = {len(train_data)} \nte_n = {len(test_data)} \nva_n = {len(val_data)}")
    return 

TTV_split(np.array([i for i in range(100000)]), 12, 4, 1)

100%|██████████| 100000/100000 [00:00<00:00, 320005.13it/s]


In [52]:
class nnModule():
    def __init__(self, params) -> None:
        self.params = params

    @abc.abstractclassmethod
    def forward(self, x):
        pass

    @abc.abstractclassmethod
    def backward(self, grad):
        pass

    def __str__(self) -> str:
        rs = ""
        strs = np.array([f"{i}: {self.params[i]}, " for i in self.params])
        for i in strs:
            rs += i
        return (f"{self.name}: (" + rs[:len(rs)-2] + ")") if self.params else self.name


class Linear(nnModule):
    def __init__(self, params) -> None:
        super().__init__(params)
        try:
            if params["in_dim"] < 1 or params["out_dim"] < 1:
                raise Exception
        except KeyError:
            raise Exception("You have to set in_dim and out_dim parameters for linear layer")
        except Exception:
            raise Exception("The in_dim and out_dim have to be greater than zero")

        self.name = "Linear"

        self.W = np.ones(params["out_dim"])
        self.B = np.zeros(params["out_dim"])

    def forward(self, x):
        return np.dot(self.W, x) + self.B

    def backward(self, grad):
        pass


class ReLU(nnModule):
    def __init__(self, params) -> None:
        super().__init__(params)


class Transformer():
    def __init__(self) -> None:
        #positional_encoder
        #multihead_attention
        #sum and norm
        #MLP
        #sum and norm
        #linear
        #softmax
        pass

In [55]:
linear = Linear(params={"in_dim": 384, "out_dim": 24})
print(linear)

Linear: (in_dim: 384, out_dim: 24)
